In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os

In [ ]:
os.chdir('./')
print(os.getcwd())
os.path.exists('licplt.jpg')

In [ ]:
img = cv.imread('./licplt.jpg') #, cv.IMREAD_GRAYSCALE)
assert img is not None, "file could not be read, check with os.path.exists()"


In [ ]:
img2 = img.copy()
template = cv.imread('template.jpg', cv.IMREAD_GRAYSCALE)
assert template is not None, "file could not be read, check with os.path.exists()"
w, h = template.shape[::-1]

In [ ]:
# All the 6 methods for comparison in a list
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
 'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
 
for meth in methods:
 img = img2.copy()
 method = eval(meth)
 
 # Apply template Matching
 res = cv.matchTemplate(img,template,method)
 min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
 
 # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
 if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
 top_left = min_loc
 else:
 top_left = max_loc
 bottom_right = (top_left[0] + w, top_left[1] + h)
 
 cv.rectangle(img,top_left, bottom_right, 255, 2)
 
 plt.subplot(121),plt.imshow(res,cmap = 'gray')
 plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
 plt.subplot(122),plt.imshow(img,cmap = 'gray')
 plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
 plt.suptitle(meth)
 
 plt.show()

In [ ]:
import cv2
import numpy as np

# Load the YOLO model (YOLOv3)
net = cv2.dnn.readNetFromDarknet('./yolov3.cfg', './yolov3.weights')

# Load the COCO class names (you can replace this with your own custom classes)
with open('./coco.names') as f:
    classes = f.read().strip().split('\n')

# Set the confidence threshold (to filter weak detections)
confidence_thresh = 0.5

# Load an image
image_path = './licplt.jpg'
image = cv2.imread(image_path)

# Create a blob from the image
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

# Set the input to the network
net.setInput(blob)

# Get the output layer names
output_layer_names = net.getUnconnectedOutLayersNames()

# Forward pass through the network
outputs = net.forward(output_layer_names)

# Process the detections
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > confidence_thresh:
            # Get bounding box coordinates
            box = detection[:4] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
            (x, y, w, h) = box.astype(int)

            # Draw bounding box and label
            color = (0, 255, 0)  # Green color
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image, classes[class_id], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Display the result
cv2.imshow('YOLO Object Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
